In [1]:
import json
import polars as pl

from solana.rpc.api import Client

### Connect to Solana client

In [2]:
# Solana public node - https://solana.com/docs/core/clusters#mainnet-beta-endpoint
client = Client("https://api.mainnet-beta.solana.com")

# Function to get the latest slot
client.is_connected()

print(f'client connected:? {client.is_connected()}')

client connected:? True


In [3]:
# get latest slot
latest_slot = client.get_slot(commitment="finalized").value
print(f"Latest slot: {latest_slot}")

# get epoch info
epoch_info = client.get_epoch_info().value
print(f"Epoch info: {epoch_info}")

Latest slot: 271422909
Epoch info: EpochInfo { epoch: 628, slot_index: 126911, slots_in_epoch: 432000, absolute_slot: 271422911, block_height: 251037620, transaction_count: Some(295104915737) }


In [4]:
# get a single block
block = client.get_block(
    slot=latest_slot, max_supported_transaction_version=0).value

In [5]:
# <class 'solders.transaction_status.UiConfirmedBlock'>
# turn into json, then turn into dictionary to read it better
block_dict = json.loads(block.to_json())
print('block:', block_dict.keys())

block: dict_keys(['previousBlockhash', 'blockhash', 'parentSlot', 'transactions', 'rewards', 'blockTime', 'blockHeight'])


### Calculate Fees and Priority Fees

In [7]:
# the fee and compute units are located in meta.
block_size = len(block_dict['transactions'])

In [8]:
block_size

1168

In [13]:
# Collect fees and compute units into lists
fees = []
compute_units = []
signatures = []
account_keys = []

# Iterate over the transactions and collect the values
for i in range(block_size):
    fee = block_dict['transactions'][i]['meta']['fee']
    units_consumed = block_dict['transactions'][i]['meta']['computeUnitsConsumed']
    block_height = block_dict['blockHeight']
    timestamp = block_dict['blockTime']
    signature = block_dict['transactions'][i]['transaction']['signatures']
    account_keys.append(block_dict['transactions'][i]['transaction']['message']['accountKeys'])
    fees.append(fee)
    compute_units.append(units_consumed)
    signatures.append(signature)

In [14]:
# Create a Polars DataFrame
df = pl.DataFrame({
    'block_number': block_height,
    'timestamp': timestamp,
    'fees': fees,
    'signatures': signatures,
    'account_keys': account_keys,
    'compute_units': compute_units
})

In [ ]:
# List of target accounts
target_accounts = [
    '96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5',
    'HFqU5x63VTqvQss8hp11i4wVV8bD44PvwucfZ2bU7gRe',
    'Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY',
    'ADaUMid9yfUytqMBgopwjb2DTLSokTSzL1zt6iGPaS49',
    'DfXygSm4jCyNCybVYYK6DwvWqjKee8pbDmJGcLWNDXjh',
    'ADuUkR4vqLUMWXxW9gh6D6L8pMSawimctcNZ5pGwDcEt',
    'DttWaMuVvTiduZRnguLF7jNxTgiMBZ1hyAumKUiL2KRL',
    '3AVi9Tg9Uo68tJfuvoKvqKNWKkC5wPdSSdeBnizKZ6jT'
]

In [20]:
# Filter the dataframe based on the account_keys column
filtered_df = df.filter(pl.col("account_keys").list.contains('96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5'))

In [21]:
filtered_df

block_number,timestamp,fees,signatures,account_keys,compute_units
i32,i32,i64,list[str],list[str],i64
251037618,1718204224,5001,"[""5KYNSZB2pXjygfjLVPxJRhbF9ULXxbrhZezxJSavtVyGjrgLnqvJNQaKgJwS2s73ekjzXxFZNrYX8qETNwaK1ecd""]","[""YubozzSnKomEnH3pkmYsdatUUwUTcm7s4mHJVmefEWj"", ""8itJ3mYW97vnbmB4ieqfwHEbEatHik4ujT8U7KKDrWmw"", … ""C9fdYbm6pohfBWHh9bfkdyum3PCPgNmpUpyMrxXpVN6m""]",55766
251037618,1718204224,5000,"[""4cwxMyGEn97VBi1B6tHdUbqbNrXcyLiqPwwX6LUj9BaFANhwFUK4mbpLohC15y7nbTQ8kj9dvamW3UTrEXs54KoV""]","[""arsc4jbDnzaqcCLByyGo7fg7S2SmcFsWUzQuDtLZh2y"", ""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"", … ""92J8nGdH9h6QNiZV35nJjqGMAGF9s2xjZ6AyJR7crf3Q""]",64465
251037618,1718204224,11295,"[""2JyhLTTkeRfMb68uZvZVU95Cb9ghvjqYAQBCtCut9rEuB2sRiM3V1mPks8nuY6fsLkbseJN2s5oSJ9nPMuDUHxCX""]","[""2h27AvjTUjh5ze39nLDxzx92Q38BV3Vv6Q2hQkjQEpKt"", ""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"", … ""11111111111111111111111111111111""]",300
251037618,1718204224,19000,"[""5pTSxMTT8XmDcbNdbQhQRSWucZUBhQs8KSR166Ki6ApvVmChcGX9HBFfzRE2ruYMHTA1YR4ZFrxRTqrZYUjagKC""]","[""Du7WvLvfz56qaz7WtwdXnL87k1fvw971fN3QogoBQmsW"", ""26MNy7mN12XvfwBYaebaiFiW6K7N7yD86rWAojCgx8Zd"", … ""TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA""]",37993
251037618,1718204224,5000,"[""5j7LLHAjCi1r5st5yXFXpDPPRt5JjCzAG2vS6UcwHpE7vyFGJnJZEdJRrHeEiBDm1qq2mUKrxDFKmBt34hzhUyKu""]","[""FnnXKnZThqCnJ49GEfby4kkedaHpi2b3NCk6eemjRKsZ"", ""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"", … ""11111111111111111111111111111111""]",300
251037618,1718204224,5000,"[""4M8VZ6JyiQvGEariMwt159SC5oqqNyfCumV5NiYCPEFUTb156eUfp3Nk5SRRejvmKZuBxWmkj4JVwGhk99o6u7qc""]","[""emhWqBszq4NzVY2E52rcB9zG3GvdNNDefqcXM2zoQjg"", ""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"", … ""MemoSq4gqABAXKb96qnH8TysNcWxMyWCqXgDLGmfcHr""]",45297
251037618,1718204224,6200,"[""42JWQr7QCKkw6dCpZtppkRqF1aAuvcJkCqZ892EKXpBhQWkQe7ZZJiba28WkmnwBP6EFPeeMtxPBuwYMQiLwR6oo""]","[""E7BHBjs179qwTKeURSWJyBW219B4oGfvoPqEmejztBku"", ""EzgznB9kXZNbpLCDoTDQrNo9vJmijAmWgETYKCngfScK"", … ""11111111111111111111111111111111""]",46113
251037618,1718204224,5000,"[""mDXPavALR2XJdq64J9CQbjMGpqBX6bfdLQmcXTyncnfaJuso4J2UFAkW4s6tv1Wy6k5xGJRCKMAaqbKwJfL5Y9A""]","[""7V184kSoKLogz8yzRto1zUxSMT6T1EiupTea5nmGNMo7"", ""96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5"", ""11111111111111111111111111111111""]",150


In [ ]:
df.with_columns(
    (pl.col('fees') - 5000).alias('priority_fee'),
    (pl.col('fees') / pl.col('compute_units')).alias('fee_per_compute_unit'),
    pl.from_epoch(pl.col('timestamp')).alias('datetime')
).sort(by='fee_per_compute_unit')

block_number,timestamp,fees,compute_units,priority_fee,fee_per_compute_unit,datetime
i32,i32,i64,i64,i64,f64,datetime[μs]
251034141,1718202627,5000,1114505,0,0.004486,2024-06-12 14:30:27
251034141,1718202627,5000,1113224,0,0.004491,2024-06-12 14:30:27
251034141,1718202627,5000,1112814,0,0.004493,2024-06-12 14:30:27
251034141,1718202627,5000,1112732,0,0.004493,2024-06-12 14:30:27
251034141,1718202627,5000,1112732,0,0.004493,2024-06-12 14:30:27
…,…,…,…,…,…,…
251034141,1718202627,403849,3759,398849,107.435222,2024-06-12 14:30:27
251034141,1718202627,405216,3759,400216,107.798883,2024-06-12 14:30:27
251034141,1718202627,416243,3759,411243,110.732376,2024-06-12 14:30:27
